In [ ]:
# flask api 사용을 위한 import
from flask import Flask, render_template, request
import sys
import cx_Oracle
import os
import pandas as pd

# flask 호출
app = Flask(__name__)

# flask 호출 방식 설정 (GET, POST 방식 둘다 가능하게끔 설정)
@app.route("/curation",  methods=['GET', 'POST'])
def index(): 
    if request.method == "POST":
        usercode = request.form["usercode"]
        res = curation(usercode)        
        return res
    elif request.method == "GET":    
        usercode = request.args.get('usercode')
        res = curation(usercode)        
        return res

# DB 접속을 위한 함수 설정 및 시스템 설정
def curation(usercode):
    # Oracle 접속
    LOCATION = r"C:\oraclexe"
    os.environ["PATH"] = LOCATION + ";" + os.environ["PATH"] #환경변수 등록
    
    # Oracle 접속을 위한 설정
    dsn=cx_Oracle.makedsn('localhost',1521,'xe')
    conn=cx_Oracle.connect('hr','a1234')
    
    cursor=conn.cursor()
    
    # Pandas 이용한 Oracle에서 사용할 정보 가져오기
    # 유저코드를 이용한 영화 관련 정보 데이터를 가져오기
    df=pd.read_sql('select usercode, moviecode, count(*) as count from ugotfilm_movie_choice where choice_date < sysdate -2/24 group by usercode, moviecode order by usercode', con=conn)
    
    # 유저 코드 설정
    user= int(usercode)
    
    # pivot을 사용한 컬럼 및 데이터 설정
    recommendation_pivot = df.pivot(index='USERCODE', columns='MOVIECODE', values='COUNT')
    # null값을 0으로 설정
    recommendation_pivot.fillna(0, inplace=True)
    
    # 비교군을 위한 함수 생성
    
    # 접속한 유저에 대한 영화 정보 관련 함수 설정
    def movie_seen(user_id):
      return recommendation_pivot.loc[user_id][recommendation_pivot.loc[user_id]>0]
    
    # 접속한 유저와 비슷한 유저를 찾기 위한 함수 설정
    def nearest_user(user_id, n):
      return small_test_corr.loc[user_id].sort_values(ascending=False)[1:n+1]
    
    # 로그인 한 유저의 선택한 영화 정보와 db에 저장된 회원의 선택 영화 정보를 비교
    # 상관관계 표시
    small_test = recommendation_pivot.T
    small_test_corr = small_test.corr()
    
    small_test_corr[user].sort_values(ascending=False)[1:11]
    res = nearest_user(user,1)
    
    # 로그인 한 유저의 정보와 비교하여 상관관계가 가장 높은 유저 코드 찾기
    # 유저코드를 찾은 후 로그인 한 유저에게 없는 영화 정보의 id값을 추출
    unseen = set(movie_seen(res.index[0]).index) - set(movie_seen(user).index)
    return str(unseen) 

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
